# Summary parsing using Flair & Classification of our characters

In this notebook, we are going to:
- cycle through all summaries in the `plot_summaries.txt` file and count the number of occurences of each character name inside
- use this data to classify the characters present inside of the `character.metadata.tsv` file into three categories:
    - **Primary:** the character name takes up over 10% of all mentioned characters
    - **Secondary:** the character name takes up less than 10% of all mentioned characters
    - **Missed:** the character name is not mentioned in the movie summary at all

### Library imports

In [57]:
import pandas as pd
import numpy as np
import ast
import missingno as msno
from geopy.geocoders import Nominatim
import geopandas as gpd
import re
import pycountry_convert as pc
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns

import dataframes as RAW

In [58]:
from flair.nn import Classifier
from flair.data import Sentence

# Load the model
tagger = Classifier.load('ner-fast')

2023-12-12 19:08:05,613 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


### Useful functions

The function `extract_character_names_flair` takes a string (which can be one of the summaries) and goes through all the words inside. If the word is considered a name, then it is appended to a list. The list of names is returned at the end of the function.

The function `count_appearances` takes a large string (it can be the summary) and a list of strings (it can be the list of characters found with the first function), and counts the number of times all strings in the list appear in the large text. The function returns a dictionary with the strings from the list and their occurence count inside of the large text.

These two functions will be used in the following way for all summaries:
- Use `extract_character_names_flair` to identify all character names inside
- Use `count_appearances` to count the number of appearances of all character names in the summary.

In [59]:
def extract_character_names_flair(summary):
    # Create a Flair Sentence
    sentence = Sentence(summary)

    # Run NER on the sentence
    tagger.predict(sentence)

    # Extract character names (NER tags labeled as PER, indicating a person)
    character_names = []

    for entity in sentence.get_spans('ner'):
        if entity.tag == 'PER':
            character_names.append(entity.text)

    return character_names

def count_appearances(larger_string, string_list):
    # Initialize an empty dictionary to store counts
    appearances_dict = {}

    # Iterate over each string in the list
    for search_string in string_list:
        # Count occurrences using the count() method (we convert to lowercase to avoid missing any occurence)
        count = larger_string.lower().count(search_string.lower())
        
        # Store the count in the dictionary
        appearances_dict[search_string] = count

    return appearances_dict

### Summary parsing

First, let us import the summary data:

In [60]:
summaries = RAW.summaries.copy()
summaries.head()

,Wiki ID,Summary
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...


Now, we can use the two functions mentionned above to cycle through all summaries and create the character appearance dictionaries:

In [61]:
# Grab a subset of the data (it takes about 8h for 10,000 summaries)
sub_summaries = summaries.iloc[:100, :].copy()

parsing_results = []

for index, row in sub_summaries.iterrows():
    # Print the index to keep track of where we are in the parsing
    print(index)

    # Extract the names from the summary
    names = set(extract_character_names_flair(row['Summary']))

    # Count the appearances of every name
    counts = count_appearances(row['Summary'], names)

    # Append the dictionary to the result list
    parsing_results.append(counts)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


The list `parsing_results` will now contain all the dictionaries from the character counting. In order to have a better wiew of the distribution of the characters in each movie summary, we can rank the dictionaries in descending order (to get the most common names at the beginning):

In [62]:
parsing_results = [
    {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)}
    for d in parsing_results
]

sub_summaries['Characters'] = parsing_results

Here is the resulting dataframe:

In [63]:
sub_summaries

,Wiki ID,Summary,Characters
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha...","{'Shlykov': 1, 'Lyosha': 1}"
1,31186339,The nation of Panem consists of a wealthy Capi...,"{'Katniss': 24, 'Peeta': 16, 'Rue': 11, 'Cato'..."
2,20663735,Poovalli Induchoodan is sentenced for six yea...,"{'Induchoodan': 18, 'Menon': 12, 'Manapally': ..."
3,2231378,"The Lemon Drop Kid , a New York City swindler,...","{'Kid': 35, 'Charley': 18, 'Moran': 8, 'Nellie..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...,"{'Lindy': 7, 'Michael': 4, 'Azaria': 4, 'Chamb..."
...,...,...,...
95,17060199,"Claire, a mild-mannered parking attendant, liv...","{'Claire': 19, 'Jay': 15}"
96,7481126,The film centres on a group of classmates who ...,"{'Sukumaran': 5, 'Murali': 3, 'Vasu': 2, 'Pius..."
97,9031450,A young woman named Hyun-chae is on train look...,"{'Hyun-chae': 4, 'Chae-hyun': 1, 'Dong-ha': 1,..."
98,1520023,Ninja Resurrection takes place in the Tokugawa...,"{'Jubei': 63, 'Shiro': 27, 'Mataemon': 16, 'Mu..."


In [68]:
sub_summaries.iloc[32,:]

Wiki ID                                                18549958
Summary       Mary was a lonely and poor child, from a famil...
Characters                                          {'Mary': 6}
Name: 32, dtype: object

In [64]:
sub_summaries.to_csv('parsing.csv', index=False)

### Character classification

Now, the goal is to classify all characters from the character data into three roles: Primary, Secondary and Missed (explained earlier). First, let us import the data and add a `Role` column (filled with NaN values):

In [65]:
characters = RAW.character_data.copy()
characters['Role'] = np.nan
characters.head()

,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,Map ID,Character ID,Actor ID,Role
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7,NaN
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4,NaN
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l,NaN
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc,NaN
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg,NaN


Now, we will classify the characters:

- First version: characters who take up over 10% of all names are primary and the rest are secondary (many characters are classified)

In [66]:
'''

for index, row in sub_summaries.iterrows():
    print(index)

    # Wiki ID of the movie to consider
    wiki_id = row['Wiki ID']

    # Dictionary of the parsing results for this movie
    parsing_result = row['Characters']

    # All characters who belong to this movie
    sub_characters = characters[characters['Wiki ID'] == wiki_id]
    
    # If the movie features actors inside of the character dataframe then proceed
    if not(sub_characters.empty):
        for i, r in sub_characters.iterrows():
            # Take one of the characters
            character = r['Character name']

            # If the considered character has a valid name then proceed
            if not(pd.isna(character)):
                # Split the character in all of its words (name, surname, etc)
                split_character_name = character.split()

                count = 0
                total = 0

                for key, value in parsing_result.items():
                    # Add all values to the total
                    total += value

                    for item in split_character_name:
                        if item in key:
                            # If we find a match then add to the count and stop (to avoid counting twice)
                            count += value
                            break
                    
                if total != 0:
                    # Compute ratio
                    ratio = count / total
                else:
                    # Empty dictionary: the character is a miss
                    ratio = 0

                if ratio > 0.1:
                    # Primary character: appears 10% of the time or more
                    characters.loc[(characters['Character name'] == character) & (characters['Wiki ID'] == wiki_id), 'Role'] = 'Primary'

                elif ratio <= 0.1 and ratio > 0:
                    # Secondary character: appears less than 10%
                    characters.loc[(characters['Character name'] == character) & (characters['Wiki ID'] == wiki_id), 'Role'] = 'Secondary'

                else:
                    # None: The character was not mentioned in the summary
                    characters.loc[(characters['Character name'] == character) & (characters['Wiki ID'] == wiki_id), 'Role'] = 'Missed'

'''

"\n\nfor index, row in sub_summaries.iterrows():\n    print(index)\n\n    # Wiki ID of the movie to consider\n    wiki_id = row['Wiki ID']\n\n    # Dictionary of the parsing results for this movie\n    parsing_result = row['Characters']\n\n    # All characters who belong to this movie\n    sub_characters = characters[characters['Wiki ID'] == wiki_id]\n    \n    # If the movie features actors inside of the character dataframe then proceed\n    if not(sub_characters.empty):\n        for i, r in sub_characters.iterrows():\n            # Take one of the characters\n            character = r['Character name']\n\n            # If the considered character has a valid name then proceed\n            if not(pd.isna(character)):\n                # Split the character in all of its words (name, surname, etc)\n                split_character_name = character.split()\n\n                count = 0\n                total = 0\n\n                for key, value in parsing_result.items():\n                

- Second version: the most common character is primary and the second most is secondary (only 2 characters are classified and there are no 'Missed' category)

In [78]:
for index, row in sub_summaries.iterrows():
    print(index)

    # Wiki ID of the movie to consider
    wiki_id = row['Wiki ID']

    # Dictionary of the parsing results for this movie
    parsing_result = row['Characters']

    # All characters who belong to this movie
    sub_characters = characters[characters['Wiki ID'] == wiki_id]

    primary = None
    secondary = None

    most = -1
    second_most = -1
    
    # If the movie features actors inside of the character dataframe then proceed
    if not(sub_characters.empty):
        for i, r in sub_characters.iterrows():
            # Take one of the characters
            character = r['Character name']

            # If the considered character has a valid name then proceed
            if not(pd.isna(character)):
                # Split the character in all of its words (name, surname, etc)
                split_character_name = character.split()

                count = 0
                total = 0

                for key, value in parsing_result.items():
                    # Add all values to the total
                    total += value

                    for item in split_character_name:
                        if item in key:
                            # If we find a match then add to the count and stop (to avoid counting twice)
                            count += value
                            break
                    
                if total != 0:
                    # Compute ratio
                    ratio = count / total
                else:
                    # Empty dictionary: the character is a miss
                    ratio = 0

                # Found a new character that appears more often than the current first
                if ratio > most:
                    # Current first becomes second
                    second_most = most
                    secondary = primary

                    # New character gats first place
                    most = ratio
                    primary = character

                else:
                    # Found a new character that appears as often as the current first and there are still no second most
                    if ratio == most and secondary == None:
                        second_most = ratio
                        secondary = character

                    # Found a new character that appears less often than the current first and more often than the current second
                    if ratio < most and ratio > second_most:
                        second_most = ratio
                        secondary = character

        # Check that the primary and secondary characters are valid and assign the role if correct
        # - if we couldn't classify both a primary and a secondary character then the movie is not useful
        # - if the character name classified as primary matches with the most appearing name in the summary then it's good
        # - same for secondary

        keys = list(parsing_result.keys())
        i = 0

        assign_primary = False
        assign_secondary = False
        
        if len(keys) >= 1 and primary != None:
            if keys[i] in primary:
                assign_primary = True

            still_in_key = (keys[i] in primary)
            while still_in_key and (i < len(keys)):
                i += 1

        if i < len(keys) and secondary != None:
            if keys[i] in secondary:
                assign_secondary = True

        if assign_primary and assign_secondary:
            characters.loc[(characters['Character name'] == primary) & (characters['Wiki ID'] == wiki_id), 'Role'] = 'Primary'
            characters.loc[(characters['Character name'] == secondary) & (characters['Wiki ID'] == wiki_id), 'Role'] = 'Secondary'

0
['Shlykov', 'Lyosha']
2
None
None
1
['Katniss', 'Peeta', 'Rue', 'Cato', 'Haymitch', 'Crane', 'Clove', 'Snow', 'Thresh', 'Seneca Crane', 'Glimmer', 'Marvel', 'Haymitch Abernathy', 'Peeta Mellark', 'Caesar Flickerman', 'Primrose Everdeen']
16
Katniss Everdeen
Peeta Mellark
0
Katniss
2
['Induchoodan', 'Menon', 'Manapally', 'Indulekha', 'Pavithran', 'Nambiar', 'Manapally Pavithran', 'Anuradha', 'Justice Menon', 'Manapally Madhavan Nambiar', 'Raman Nair', 'Chandrabhanu', 'Mooppil Nair', 'Saikumar', 'Justice Maranchery Karunakara Menon', 'Moopil Nair', 'Ramakrishnan', 'Manapally Sudheeran', 'Poovalli Induchoodan', 'Aishwarya', 'Jayakrishnan', 'DYSP Sankaranarayanan', 'Kanaka', 'Nandagopal Maarar']
24
Marancheri Induchoodan
M.K. Menon
0
Induchoodan
3
['Kid', 'Charley', 'Moran', 'Nellie', 'Brainy', 'Nellie Thursday', 'Moose Moran', 'Oxford" Charley', 'Henry', 'Brainy Baxter']
10
None
None
4
['Lindy', 'Michael', 'Azaria', 'Chamberlains', 'Michael Chamberlain']
5
Lindy Chamberlain
Michael Cham

The characters who still have a NaN value inside of their `Role` column are characters who are not featured inside of their summaries, so they will not be useful. Therefore, we filter the characters who were assigned a role:

In [81]:
result = characters[characters['Role'].notna()]
result

,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,Map ID,Character ID,Actor ID,Role
4633,20663735,/m/051zjwb,2000,M.K. Menon,1935-12-10,M,NaN,/m/0dryh9k,Thilakan,64.0,/m/059t6pp,/m/0h73lnb,/m/02hkvw,Secondary
4639,20663735,/m/051zjwb,2000,Marancheri Induchoodan,1960-05-21,M,1.720,/m/0dryh9k,Mohanlal,39.0,/m/059t6p_,/m/0h8gtfl,/m/02fbpz,Primary
107615,595909,/m/02tqm5,1988-11-03,Michael Chamberlain,1947-09-14,M,1.822,/m/02jvpv,Sam Neill,41.0,/m/02tbjj2,/m/0h2qv0j,/m/01ckhj,Secondary
107616,595909,/m/02tqm5,1988-11-03,Lindy Chamberlain,1949-06-22,F,1.680,NaN,Meryl Streep,39.0,/m/02tb1h6,/m/05z0x_h,/m/0h0wc,Primary
128473,2462689,/m/07ftxt,1941-05-03,Ann Mitchell,1907-07-16,F,1.650,NaN,Barbara Stanwyck,33.0,/m/0k0kbm,/m/0h57bdz,/m/0bw6y,Secondary
128474,2462689,/m/07ftxt,1941-05-03,Long John Willoughby - 'John Doe',1901-05-07,M,1.905,NaN,Gary Cooper,39.0,/m/0k0kbg,/m/0h2svgl,/m/0c2tf,Primary
222412,4596602,/m/0cbs4j,1999-10-11,Gabriel Darbeaux,1964-05-16,M,NaN,NaN,Wayne Gerard Trotman,35.0,/m/0l7pwhl,/m/0l7pwhp,/m/03cckf8,Primary
222419,4596602,/m/0cbs4j,1999-10-11,Valentino Tarontola,NaN,M,NaN,NaN,Chris Gabriel,NaN,/m/0l7qvc1,/m/0l7qvc4,/m/0l7qvcc,Secondary
299941,447194,/m/029jt9,1949-12-21,Delilah,1913-11-09,F,1.700,/m/0ffj5g,Hedy Lamarr,36.0,/m/03jpf27,/m/0h6gc84,/m/016yqj,Secondary
299942,447194,/m/029jt9,1949-12-21,Samson,1913-01-29,M,1.880,NaN,Victor Mature,36.0,/m/03jslrf,/m/0h6gc8f,/m/029135,Primary


Finally, we store the result inside of a CSV file:

In [ ]:
result.to_csv('character_classification.csv', index=False)